# Building ANCOR Dataset

In [ ]:
from french_crs.ancor2dataset import dataset_builder

# Do-able with two nested loops
# for method in ["balanced", "representative", "window"]:
#     for subcorpus in [["INDIRECTE"],["DIRECTE"],["ANAPHORE"]]:

dataset_params={
    # There are other three possible strategies: (1)"balanced", (2)"representative" and (3)"window"
    "strategy" : "balanced,
    "ancor_corpus_path" : "../DISTRIB_ANCOR/",
    # All possibilities are: ["corpus_OTG","corpus_UBS","corpus_ESLO" ,"corpus_ESLO_CO2"]
    "sub_corpus_filter" : ["corpus_OTG","corpus_UBS","corpus_ESLO" ,"corpus_ESLO_CO2"], 
    # All possibilities are: ["DIRECTE", "INDIRECTE", "ANAPHORE"]
    "coreference_type_filter" : ["DIRECTE"], 
    "dataset_output_folder" : "../datasets/",
    # This will be used only in window strategy
    "window_size" : 5 
}

dataset = dataset_builder(**dataset_params)
dataset.build_dataset(file_analysis_alert=True)
dataset.merge_dataset(delete_original_after_merge=False)

# Dataset Splitting

In [ ]:
# Two ways of Training: (1) commented codes down with parameter.
#                       (2) uncommented with mentioned explicity the config file.

from french_crs.model_training import dataset_splitter

# ds1=dataset_splitter("../datasets/balanced_Full/balanced_Full_Merged.xlsx",
#                     "../datasets/balanced_Full/balanced_Full_Train.xlsx",
#                     "../datasets/balanced_Full/balanced_Full__Test.xlsx",
#                       split_config_json="../datasets/split_config.json")

# ds1.dataset_splitter_by_file(lower_rate=0.20,upper_rate=0.21, files_num=30)


ds2=dataset_splitter("../datasets/window_INDIRECTE/window_INDIRECTE_Merged.xlsx",
                    "../datasets/window_INDIRECTE/window_INDIRECTE_Train.xlsx",
                    "../datasets/window_INDIRECTE/window_INDIRECTE_Test.xlsx",
                      split_config_json="../datasets/split_config.json")

dict_files=ds2.dataset_splitter_by_json_config()


# Model Training

In [43]:
from french_crs.model_training import model_trainer

"""
12 possibilities for "train" and "test" variables

balanced_ANAPHORE
balanced_DIRECTE
balanced_Full
balanced_INDIRECTE
representative_ANAPHORE
representative_DIRECTE
representative_Full
representative_INDIRECTE
window_ANAPHORE
window_DIRECTE
window_Full
window_INDIRECTE
"""

train="balanced_INDIRECTE"
test="representative_INDIRECTE"

model=model_trainer( "../datasets/"+ train+"/"+train+"_Train.xlsx",
                     "../datasets/"+ test+"/"+test+"_Test.xlsx",
                     "../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                     "IS_CO_REF",
                     "IS_CO_REF"
                   )

model.columns_drop_list = ['m1_DEF', 'm2_DEF', 'ID_DEF',
                           'DISTANCE_MENTION','DISTANCE_WORD',
                           'DISTANCE_CHAR']

model.convert_columns_to_numeric()
print("Model_Name :",train+".model\n")
print("Train Dataset :",train+"_Train.xlsx")
print("Test  Dataset :",test+"_Train.xlsx")
model.train_model_random_forest(model_name="../pre-trained language models/Model_ANCOR_"+train+".model",max_depth=10, random_state=0,n_estimators=250)

Model_Name : balanced_INDIRECTE.model

Train Dataset : balanced_INDIRECTE_Train.xlsx
Test  Dataset : representative_INDIRECTE_Train.xlsx


{'precision_score_pos': 0.37110481586402266,
 'recall_score_pos': 0.8791946308724832,
 'f1_score_pos': 0.5219123505976095,
 'precision_score_neg': 0.9849624060150376,
 'recall_score_neg': 0.841541755888651,
 'f1_score_neg': 0.9076212471131639}

# SCORCH Chains Building

In [37]:
from french_crs.pairs2chains import chains_builder

model_chains=chains_builder(path_gold_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx", 
                            path_model_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                            gold_column="IS_CO_REF",
                            model_column="Prediction",
                            scorch_output_path="../",
                            threshold=0.5)

model_chains.generate_gold_model_json_output(mode="train")
os.popen('echo Returned output')

SCORCH json formats saved at following addresses:
/Users/mehdi.mirzapour/French-CRS/coref_chains_gold.json
/Users/mehdi.mirzapour/French-CRS/coref_chains_pred.json


# SCORCH Outcome

In [93]:
import os
bashCommand = "scorch coref_chains_gold.json coref_chains_pred.json > ../mm.txt"
os.system(bashCommand)
f = open("../mm.txt",'r')
message = f.read()
print(message)

MUC:	R=0.8196010407632264	P=0.8931947069943289	F₁=0.8548168249660787
B³:	R=0.7818069604042842	P=0.8504910598059741	F₁=0.8147039597980739
CEAF_m:	R=0.7387440758293838	P=0.8280212483399735	F₁=0.7808390732623669
CEAF_e:	R=0.6954840434955073	P=0.830544560870751	F₁=0.7570375651477038
BLANC:	R=0.697837406779154	P=0.8307640432084871	F₁=0.7577377208853988
CoNLL-2012 average score: 0.8088527833039522



#  Model Testing

In [ ]:
from model_training import model_tester

model_parameter={
                "model_name" : "./Models/Random_Forest_(Normal)_OTG_Neg_90_Pos_10.model",
                "input_file" : "./Datasets/corpus_ALL_Window_30_Test.xlsx",
                "output_file" : "./Datasets/corpus_ALL_Window_30_Test_Called_Seperately.xlsx",
                "column_gold" : "IS_CO_REF",
                "column_outcome" : "Prediction",
                "threshold" : 0.5
                }

model=model_tester(**model_parameter)
model.apply_model_to_dataset()